In [184]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lxml.html as lh
from IPython.display import clear_output
from datetime import datetime
import json
%matplotlib inline

In [3]:
# want the urls, so need more complexity
# opsTable = pd.read_html('https://www.canada.ca/en/department-national-defence/services/military-history/history-heritage/past-operations.html')[0]

In [4]:
page = requests.get('https://www.canada.ca/en/department-national-defence/services/military-history/history-heritage/past-operations.html')

In [5]:
content = BeautifulSoup(page.text, 'html.parser')

In [6]:
table = content.find('div',{'class': 'table-responsive'}).findAll('tr')

In [7]:
base = 'https://www.canada.ca'
arr = []
for i in range(len(table)):
    colText = []
    cols = table[i].findAll('td')
    for j in range(len(cols)):
        colText.append(cols[j].text)
        if j == 4:
            colText.append(base + cols[0].find('a').get('href'))
    if i == 0:
        colText = colText = ['opName','countryRegion','continent','startDate','endDate','url']
    arr.append(colText)

In [17]:
df = pd.DataFrame(arr)
header = df.iloc[0]
df = df[1:]
df.columns = header

In [18]:
df.startDate = pd.to_datetime(df.startDate)
df.endDate = pd.to_datetime(df.endDate,errors='coerce',)

In [19]:
df.loc[df.endDate.isnull(),'endDate'] = datetime.now()

In [20]:
df.reset_index(inplace=True,drop=True)

In [21]:
df.head(5)

,opName,countryRegion,continent,startDate,endDate,url
0,(Algeria 1980),Algeria,Africa,1980-11-14,1980-11-19,https://www.canada.ca/en/department-national-d...
1,PASTEL,Angola,Africa,1991-05-30,1995-02-08,https://www.canada.ca/en/department-national-d...
2,PRUDENCE,Central African Republic,Africa,1998-04-15,1999-10-22,https://www.canada.ca/en/department-national-d...
3,MALLARD,Congo,Africa,1960-07-14,1964-06-30,https://www.canada.ca/en/department-national-d...
4,Opération des Nations unies au Congo,Congo,Africa,1960-07-14,1964-06-30,https://www.canada.ca/en/department-national-d...


In [183]:
bigArr = []
for i in range(len(df)):
    clear_output(wait=True)
    print('{}/{}'.format(i+1,len(df)+1))
    
    rowDict = {}
    row = df.iloc[i]
    rowDict['opName'] = row.opName
    rowDict['countryRegion'] = row.countryRegion
    rowDict['continent'] = row.continent
    rowDict['startDate'] = row.startDate.strftime('%Y-%m-%d')
    rowDict['endDate'] = row.endDate.strftime('%Y-%m-%d')
    rowDict['url'] = row.url
    rowDict['details'] = {}
    
    page = requests.get(row.url)
    content = BeautifulSoup(page.text, 'html.parser')
    area = content.find('div',{'class':'mwsbodytext text parbase section'}).findAll('p')

    for i in range(len(area)):
        current = area[i]
        # sometimes there's a preamble description that really adds nothing
        if (i == 0) and (current.find('strong') is None):
            pass
        elif current.find('strong') is not None:
            thisText = current.text.replace(u'\xa0',u'').split(':',1)
            if (len(thisText) > 1):
                rowDict['details'][thisText[0]] = thisText[1]
            else:
                rowDict['details'][thisText[0]] = 'NONE'
        else:
            rowDict['details'][thisText[0]] += ' ' + current.text
    
    bigArr.append(rowDict)

335/336


In [185]:
with open('canmil.json', 'w') as outfile:
    json.dump(bigArr,outfile)